<a href="https://colab.research.google.com/github/Dartpixel/fakeNewsClassifierUsingLSTM/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [2]:
import pandas as pd

In [6]:
df=pd.read_csv('/content/drive/MyDrive/dataset/train.csv')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
###Drop Nan Values
df=df.dropna()


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.15.0'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3258, 4972, 3495, 3861, 4312, 522, 2058, 1822, 1339, 376],
 [2676, 4674, 3127, 4579, 4090, 2427, 1987],
 [2155, 1876, 2822, 4485],
 [1248, 1872, 3213, 3345, 96, 3385],
 [437, 4090, 3254, 2575, 1192, 2562, 4090, 3663, 4273, 2625],
 [136,
  4682,
  1268,
  3510,
  1294,
  3346,
  1886,
  3425,
  2825,
  4679,
  2615,
  3042,
  1860,
  2981,
  1987],
 [11, 2272, 4791, 985, 1145, 92, 859, 2534, 3440, 3603, 1730],
 [4096, 2356, 4408, 1221, 4300, 3439, 3346, 2535, 3440, 3603, 1730],
 [2225, 908, 3206, 2416, 2585, 4273, 166, 1400, 3346, 746],
 [1528, 4183, 3005, 2042, 3419, 4231, 1755, 1468],
 [1709, 3364, 705, 3998, 1164, 4593, 1492, 4958, 1700, 481, 250],
 [3345, 273, 4312, 4273, 3346, 4300],
 [2412, 578, 4597, 3247, 2111, 45, 544, 1320, 3136],
 [398, 2370, 3997, 2616, 614, 2946, 4285, 3440, 3603, 1730],
 [4874, 2902, 448, 4956, 4700, 3440, 3603, 1730],
 [3081, 2804, 561, 361, 4836, 3189, 4754, 847, 431, 2788],
 [1121, 3135, 4674],
 [3545, 4820, 925, 2238, 3346, 2015, 3874, 1987],
 [3176,

### Embedding Representation

In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1822 1339  376]
 [   0    0    0 ... 4090 2427 1987]
 [   0    0    0 ... 1876 2822 4485]
 ...
 [   0    0    0 ... 3440 3603 1730]
 [   0    0    0 ...  596 4425  767]
 [   0    0    0 ... 4256  262 2749]]


In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3258,
       4972, 3495, 3861, 4312,  522, 2058, 1822, 1339,  376], dtype=int32)

In [27]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [28]:
len(embedded_docs),y.shape

(18285, (18285,))

In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [32]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 59ms/step - loss: 0.3362 - accuracy: 0.8429 - val_loss: 0.1944 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 10s 52ms/step - loss: 0.1399 - accuracy: 0.9451 - val_loss: 0.2013 - val_accuracy: 0.9163
Epoch 3/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0991 - accuracy: 0.9631 - val_loss: 0.2653 - val_accuracy: 0.9173
Epoch 4/10
192/192 [==============================] - 11s 57ms/step - loss: 0.0751 - accuracy: 0.9727 - val_loss: 0.2785 - val_accuracy: 0.9140
Epoch 5/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0548 - accuracy: 0.9807 - val_loss: 0.2885 - val_accuracy: 0.9128
Epoch 6/10
192/192 [==============================] - 11s 55ms/step - loss: 0.0402 - accuracy: 0.9865 - val_loss: 0.3775 - val_accuracy: 0.9118
Epoch 7/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0257 - accuracy: 0.9917 - val_loss: 0.4320 - val_accuracy: 0.

### Adding Dropout

In [46]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [48]:
#y_pred = model.predict_classes(X_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 2s 9ms/step


In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
confusion_matrix(y_test,y_pred)

array([[ 927, 2492],
       [1592, 1024]])

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.3232808616404308